# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 8 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.

### Exercise 8.1 

**What would the oracle operation $U_f$ be for 𝑓 if 𝑓 was `zero`?**

Let's work it out one step at a time.
- From the definition for $U_f$, we know that $U_f \left|xy\right\rangle = \left|x\right\rangle \left|y \oplus f(x)\right\rangle$.
- Substituting in `zero` for 𝑓, 𝑓(𝑥) = 0, we get that $U_f \left|xy\right\rangle = \left|x\right\rangle\left|y \oplus 0\right\rangle$.
- We can use that 𝑦 ⊕ 0 = 𝑦 to simplify this a little bit further, getting that $U_f \left|xy\right\rangle = \left|x\right\rangle\left|y\right\rangle$.
- At this point, we notice $U_f$ does nothing to its input state, so we can implement it by... doing nothing.

----
### Epilogue

_The following cell logs what version of the components this was last tested with._

In [1]:
%version

Component,Version
iqsharp,0.17.2105143879
Jupyter Core,1.5.0.0
.NET Runtime,".NETCoreApp,Version=v3.1"
